# Imports and libraries

In [1]:
!pip install tensorflow tensorflow-gpu opencv-python matplotlib moviepy

Defaulting to user installation because normal site-packages is not writeable
  Using cached tensorflow-gpu-2.12.0.tar.gz (2.6 kB)
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [18 lines of output]
      Traceback (most recent call last):
        File "<string>", line 2, in <module>
        File "<pip-setuptools-caller>", line 34, in <module>
        File "/tmp/pip-install-x76z19dl/tensorflow-gpu_91e8ddca6a034447b3e172319b8b6503/setup.py", line 37, in <module>
          raise Exception(TF_REMOVAL_WARNING)
      Exception:
      
      The "tensorflow-gpu" package has been removed!
      
      Please install "tensorflow" instead.
      
      Other than the name, the two packages have been identical
      since TensorFlow 2.1, or roughly since Sep 2019. For more
      information, see: pypi.org/project/tensorflow-gpu
      
      
      [end of output]
  
  note: This er

In [2]:
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
import cv2
import moviepy.editor as mp
import pygame
import time
import threading

2023-08-24 18:40:55.895911: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-08-24 18:40:55.897075: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-24 18:40:55.924530: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-24 18:40:55.925202: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-24 18:40:56.484637: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

# Load model

In [4]:
interpreter = tf.lite.Interpreter(model_path='models/lite-model_movenet_singlepose_lightning_3.tflite')
interpreter.allocate_tensors()

# Input and output details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


# Global variables

In [5]:
EDGES = {
    (0, 1): 'm',
    (0, 2): 'c',
    (1, 3): 'm',
    (2, 4): 'c',
    (0, 5): 'm',
    (0, 6): 'c',
    (5, 7): 'm',
    (7, 9): 'm',
    (6, 8): 'c',
    (8, 10): 'c',
    (5, 6): 'y',
    (5, 11): 'm',
    (6, 12): 'c',
    (11, 12): 'y',
    (11, 13): 'm',
    (13, 15): 'm',
    (12, 14): 'c',
    (14, 16): 'c'
}

# Helper functions

In [6]:
def predict(frame):
    # Reshape image
    img = frame.copy()
    img = tf.image.resize_with_pad(np.expand_dims(img, axis=0), 192,192)
    input_image = tf.cast(img, dtype=tf.float32)
        
    # Make predictions 
    interpreter.set_tensor(input_details[0]['index'], np.array(input_image))
    interpreter.invoke()
    return interpreter.get_tensor(output_details[0]['index'])

In [7]:
def draw_keypoints(frame, keypoints, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for kp in shaped:
        ky, kx, kp_conf = kp
        if kp_conf > confidence_threshold:
            cv2.circle(frame, (int(kx), int(ky)), 4, (0,255,0), -1) 

In [8]:
def draw_connections(frame, keypoints, edges, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for edge, color in edges.items():
        p1, p2 = edge
        y1, x1, c1 = shaped[p1]
        y2, x2, c2 = shaped[p2]
        
        if (c1 > confidence_threshold) & (c2 > confidence_threshold):      
            cv2.line(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0,0,255), 2)

# Make detections

In [9]:
video_path = 'songs/copines.mp4'

In [10]:
# Capture all video frames
preds = []

cap = cv2.VideoCapture(video_path)

while cap.isOpened():
    ret, frame = cap.read()

    if ret == False:
        break
    
    frame = cv2.flip(frame,1)
    
    keypoints_with_scores = predict(frame)
    
    preds.append(keypoints_with_scores[0][0])
    # Rendering 
    #draw_connections(frame, keypoints_with_scores, EDGES, 0.4)
    #draw_keypoints(frame, keypoints_with_scores, 0.4)
        
cap.release()
#cv2.destroyAllWindows()

print(len(preds))

2023-08-24 18:41:28.767182: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:268] failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error
2023-08-24 18:41:28.767210: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:168] retrieving CUDA diagnostic information for host: VAAL-ubu
2023-08-24 18:41:28.767214: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:175] hostname: VAAL-ubu
2023-08-24 18:41:28.767331: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:199] libcuda reported version is: 535.86.5
2023-08-24 18:41:28.767344: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:203] kernel reported version is: 535.86.5
2023-08-24 18:41:28.767347: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:309] kernel version seems to match DSO: 535.86.5


657


In [11]:
clip = mp.VideoFileClip(video_path)
clip = clip.resize((400, 640))
clip = mp.vfx.mirror_x(clip)

def play_clip():
    #time.sleep(0)
    clip.preview()
    pygame.quit()

In [18]:
webcam_preds = []

clip_thread = threading.Thread(target=play_clip, args=())

#Start the stopwatch
start_time = time.time()

webcam_cap = cv2.VideoCapture(0, cv2.CAP_V4L)
time.sleep(1)
clip_thread.start()

print(webcam_cap.get(cv2.CAP_PROP_FPS))

while webcam_cap.isOpened():
    time_elapsed = time.time() - start_time

    if time_elapsed > clip.duration + 1.0:
        break

    # if len(webcam_preds) >= len(preds):
    #     break
    
    ret, frame = webcam_cap.read()

    frame = cv2.flip(frame,1)
    
    keypoints_with_scores = predict(frame)
    
    webcam_preds.append(keypoints_with_scores)

    # Rendering 
    # draw_connections(frame, keypoints_with_scores, EDGES, 0.4)
    # draw_keypoints(frame, keypoints_with_scores, 0.4)
    
    #cv2.imshow('MoveNet Lightning', cv2.resize(frame, (480, 640)))
    
    # if cv2.waitKey(1) & 0xFF==ord('q'):
    #     break
        
cap.release()
cv2.destroyAllWindows()


30.0


In [19]:
print(len(preds))
print(len(webcam_preds))

657
332
